# lab1_2

In [193]:
import math
import numpy as np
import sympy as sm
from sympy import Derivative, Dummy
from sympy.core.evaluate import evaluate
from itertools import groupby, zip_longest
import matplotlib.pyplot as plt
from tabulate import tabulate
from pprint import pprint

from IPython.display import display, Latex


In [194]:
sm.init_printing(use_latex='mathjax')
x, y, c = sm.symbols('x, y, c', real=True)
C1, C2 = sm.symbols("C1, C2", real=True)
c = sm.symbols("c")
u = sm.Function('u')
ux = u(x)
ux1 = Derivative(u(x), x)
ux2 = Derivative(u(x), (x, 2))

h, uup1, uum1, uu0 = sm.symbols('h, u_i+1, u_i-1, u_i')
uux1 = (uup1 - uum1)/(2*h)
uux2 = (uup1 - 2*uu0 + uum1)/(h**2)

In [195]:
zad1 = {
    'p': sm.sqrt(1 + x**2)*0.4,
    'q': 4*(1 + x**2),
    'f': 20*sm.exp(-x),
    'a': 0, 'UA': 0,
    'b': 2.5, 'UB': 0,
    'E': 0.05
}
zad0 = {
    'p': 1/x,
    'q': 0.5,
    'f': 0.5*x**2 - sm.ln(x) + 4,
    'a': 1, 'UA': 1,
    'b': 2, 'UB': 1.1137,
    'E': 0.05
}

In [196]:
mainEq = sm.Eq(ux2 + zad1['p']*ux1 + zad1['q']*ux, zad1['f'])
mainEq

       ________                                2               
      ╱  2      d          ⎛   2    ⎞         d              -x
0.4⋅╲╱  x  + 1 ⋅──(u(x)) + ⎝4⋅x  + 4⎠⋅u(x) + ───(u(x)) = 20⋅ℯ  
                dx                             2               
                                             dx                

In [197]:
#apro = mainEq.subs({ux: uu0, ux2: uux2, ux1: uux1 })
apro = sm.Eq(uux2 + zad0['p']*uux1 + zad0['q']*uu0,  zad0['f'])
apro

         uᵢ₊₁ - uᵢ₋₁   -2⋅uᵢ + uᵢ₊₁ + uᵢ₋₁        2             
0.5⋅uᵢ + ─────────── + ─────────────────── = 0.5⋅x  - log(x) + 4
            2⋅h⋅x                2                              
                                h                               

In [198]:
n = 2  # n=6
hh = (zad0['b'] - zad0['a']) / n
display(hh)
us = sm.symbols(' '.join([f"u{i}" for i in range(n+1)]))
us

0.5

(u₀, u₁, u₂)

In [199]:
list_x = [zad0['a'] + hh*i for i in range(n+1)]
list_x

[1.0, 1.5, 2.0]

In [200]:
tyts = [apro.subs({x: list_x[i], h: hh, uum1: us[i-1], uu0: us[i], uup1: us[i+1]})  for i in range(1,n)]
tyts.append(sm.Eq(us[0] + (-us[2]+4*us[1]-3*us[0])/(2*hh), zad0['UA']))
tyts.append(sm.Eq(us[n] - 0.5*(3*us[n]-4*us[n-1]+us[n-2])/(2*hh), zad0['UB']))

#tyts = [sm.Eq(2*us[0] + 1*us[1] + 0*us[2], 3), sm.Eq(5*us[0] + 4*us[1] + 2*us[2], 6), sm.Eq(0*us[0] + 1*us[1] - 3*us[2], 2)]
display(*tyts)

3.33333333333333⋅u₀ - 7.5⋅u₁ + 4.66666666666667⋅u₂ = 4.71953489189184

-2.0⋅u₀ + 4.0⋅u₁ - 1.0⋅u₂ = 1

-0.5⋅u₀ + 2.0⋅u₁ - 0.5⋅u₂ = 1.1137

In [201]:
result = sm.solve(tyts, us)

In [202]:
display(*list(map(lambda e: [e[0], e[1].n(4)], result.items())))

[u₀, 1.227]

[u₁, 1.5]

[u₂, 2.545]

In [203]:
def maper(expr):
    if (expr.func == sm.Mul):
        return [expr]
    return list(expr.args)

def maper2(expr):
    if(isinstance(expr, int) or isinstance(expr, float)):
        return float(expr)
    if (expr.func == sm.Symbol):
        return 1.0
    elif (expr == 0):
        return 0.0
    return float(expr.args[0])

def comparator(expr):
    if (expr.args == () ):
        text = str(expr)
    else:
        text = str(expr.args[1])
    return int(text[1:])

express = list(map(lambda t: maper(t.lhs), tyts))

for e in express:
    e.sort(key=comparator)
for e in express:
    if (len(e) != n+1):
        for i, v in enumerate(e):
            if comparator(v) != i:
                e.insert(i, 0)
        if (len(e) != n+1):
            e.extend([0]*(n+1-len(e)))
print(express)        

expr_values = list(map(lambda t: list(map(lambda t2: maper2(t2), t)), express))
f_values = list(map(lambda t: float(t.rhs), tyts))
print(tabulate(expr_values, floatfmt=".3f"))
print(f_values)

[[3.33333333333333*u0, -7.5*u1, 4.66666666666667*u2], [-2.0*u0, 4.0*u1, -1.0*u2], [-0.5*u0, 2.0*u1, -0.5*u2]]
------  ------  ------
 3.333  -7.500   4.667
-2.000   4.000  -1.000
-0.500   2.000  -0.500
------  ------  ------
[4.719534891891835, 1.0, 1.1137]


In [204]:
count = n+1
ptable = np.eye(count)
ptable[0,0:2] = expr_values[0][:2]

for i, e in enumerate(expr_values[:-2], 1):
    ptable[i,i-1:i+2] = expr_values[i]
ptable[count-1,count-2:count] = expr_values[count-1][1:]
print(tabulate(ptable))

--------  ----  ----
 3.33333  -7.5   0
-2         4    -1
 0         2    -0.5
--------  ----  ----


In [205]:
Y, A, B = [], [], []
Y = ptable[0,0]

def TDMA(a,b,c,f):
    a, b, c, f = map(lambda k_list: list(map(float, k_list)), (a, b, c, f))
    a[0] = 0
    n = len(f)
    c[n-1] = 0
    alpha = [0]*n
    beta = [0]*n
    x = [0]*n
    u = [0]*n
    
    x[0] = b[0]
    alpha[0] = -c[0]/x[0]
    beta[0] = f[0]/x[0]
    
    for i in range(1,n-1):
        x[i] = b[i] + a[i]*alpha[i-1]
        alpha[i] = -c[i]/x[i]
        beta[i] = (f[i] - a[i]*beta[i-1])/x[i]
    
    x[n-1] = b[n-1] + a[n-1]*alpha[n-2]
    beta[n-1] = (f[n-1] - a[n-1]*beta[n-2])/x[n-1]
    
    u[n-1] = beta[n-1]
    for i in range(n-2,-1,-1):
        u[i] = alpha[i]*u[i+1] + beta[i]
    

    return u

a = [0] + [expr_values[i+1][i] for i in range(n+1-1)]
b = [expr_values[i][i] for i in range(n+1)]
c = [expr_values[i][i+1] for i in range(n+1-1)] + [0]

print(tabulate([a,b,c]))
xxx = TDMA(*[a,b,c], f_values)
display(*xxx)

--------  --  ----
 0        -2   2
 3.33333   4  -0.5
-7.5      -1   0
--------  --  ----


0.6137000000000006

-0.35651576336335555

-3.6534630534534234

In [206]:
display(*list(map(lambda e: [e[0], e[1].n(4)], result.items())))

[u₀, 1.227]

[u₁, 1.5]

[u₂, 2.545]